In [2]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [3]:
# import dataset
dataset = pd.read_csv('csv/cleaned_train_all.csv')


x = dataset.drop(['credit_card_default'], axis = 1)
y = dataset['credit_card_default'].values
print(x.shape, y.shape)

(43508, 16) (43508,)


In [5]:
scalar =  preprocessing.StandardScaler()

# copy of datasets
X_train = x.copy()

# numerical features

#for all 
num_cols = ['net_yearly_income','no_of_days_employed','yearly_debt_payments','credit_limit']

#for less
#num_cols = ["credit_limit_used(%)", "credit_score"]

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = scalar.fit(X_train[[i]])
    
    # transform the training data column
    X_train[i] = scale.transform(X_train[[i]])

X_train = X_train.set_index('customer_id')
X_train.head()

,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
customer_id,,,,,,,,,,,,,,,
CST_115179,46,0,0.0,1,0.0,-0.135900,-0.480983,1.0,1.0,0.072774,-0.163817,73,544,2,1
CST_121920,29,1,0.0,1,0.0,-0.133081,-0.465489,2.0,0.0,-0.954202,-0.038524,52,857,0,0
CST_109330,37,1,0.0,1,0.0,0.042769,-0.483912,2.0,0.0,0.961142,-0.013187,43,650,0,0
CST_128288,39,0,0.0,1,0.0,-0.114861,-0.399679,2.0,0.0,-0.534813,-0.072174,20,754,0,0
CST_151355,46,1,1.0,1,0.0,0.272478,-0.474905,1.0,0.0,0.374525,0.061461,75,927,0,0


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size = 0.25, random_state = 0)

In [7]:
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]


In [8]:
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

grid_result = grid_search.fit(X_train, y_train)

In [9]:
y_pred_LG =grid_result.predict(X_test)

#Parameter setting that gave the best results on the hold out data.
print(grid_search.best_params_ ) 
#Mean cross-validated score of the best_estimator
print('Best Score - KNN:', grid_search.best_score_ )

print('F1 Score - KNN:', metrics.f1_score(y_test, y_pred_LG)) 

print('Accuracy Score - KNN ', metrics.accuracy_score(y_test, y_pred_LG)) 


{'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score - KNN: 0.9801415786076408
